In [1]:
import sys
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.metrics import classification_report
from keras.models import load_model
import cv2

In [2]:
from flask import request
from flask import jsonify
from flask import Flask

In [3]:
df = pd.read_pickle('semicon_wafer_final.pkl')

In [4]:
app = Flask(__name__)

In [5]:
def get_model():
    global model
    model = load_model('model_interarea_intercubic.h5')
    print(" * Model loaded!")

In [6]:
def preprocess_image(image):
    if image.shape != (40, 40):
        if image.size >= 1600:
            image = cv2.resize(image, dsize=(40, 40), interpolation=cv2.INTER_AREA)
        elif image.size < 1600:
            image = cv2.resize(image, dsize=(40, 40), interpolation=cv2.INTER_CUBIC)
    image = image.reshape((1, *image.shape, 1))
    return image

In [7]:
print(" * Loading Keras model...")
get_model()

 * Loading Keras model...
 * Model loaded!


In [8]:
@app.route("/predict", methods=["POST"])
def predict1(image):
    processed_image = preprocess_image(image)
    
    prediction = model.predict(processed_image).tolist()

    response = {
        'prediction': {
            'Center': prediction[0][0],
            'Donut': prediction[0][1],
            'Edge-Loc': prediction[0][2],
            'Edge-Ring': prediction[0][3],
            'Loc': prediction[0][4],
            'Random': prediction[0][5],
            'Scratch': prediction[0][6],
            'None': prediction[0][7],
        }
    }
    return jsonify(response)